In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from PIL import Image

# Definiere den Pfad zu deinem Datensatz
data_path = 'images/archive/bloodcells_dataset'

# Laden des ResNet50-Modells als Basis
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Einfrieren der Gewichte des Basismodells
for layer in base_model.layers:
    layer.trainable = False

# Erstellen des Transfer-Learning-Modells
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(8, activation='softmax')  # Annahme: 8 Klassen für die Blutzellenarten
])

# Kompilieren des Modells
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Erstellen von Daten-Generatoren für das Training und die Validierung
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Trainingsdaten-Generator
train_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    interpolation='bilinear',  # Optional: Du kannst die Interpolation für die Bildgrößenänderung festlegen
)

# Validierungsdaten-Generator
validation_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    interpolation='bilinear',
)



Found 13677 images belonging to 8 classes.
Found 3415 images belonging to 8 classes.


In [ ]:
# Trainieren des Modells
model.fit(train_generator, epochs=1, validation_data=validation_generator, verbose=2)

In [ ]:
# Beispiel zum Laden und Vorhersagen eines einzelnen Bildes
example_image_path = 'Pfad/zum/Ihr/Bild.jpg'  # Passe den Pfad an
example_image = Image.open(example_image_path)
example_image = example_image.resize((224, 224), resample=Image.BILINEAR)  # Bildgröße anpassen
example_image_array = img_to_array(example_image)
example_image_array = example_image_array.reshape((1, 224, 224, 3))  # Batch-Dimension hinzufügen

# Vorhersage für das Beispielbild
prediction = model.predict(example_image_array)
print("Vorhersage für das Beispielbild:", prediction)